In [1]:
!pip install sagemaker --upgrade
!pip install boto3 --upgrade
!pip install botocore --upgrade
# '2.203.1', '1.34.19', '1.34.19'

In [2]:
import sagemaker
import boto3
import botocore
sagemaker.__version__, boto3.__version__, botocore.__version__
# ('2.203.1', '1.34.19', '1.34.19')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


('2.204.0', '1.34.26', '1.34.26')

In [50]:
import sagemaker
from sagemaker.serializers import CSVSerializer, JSONSerializer, JSONLinesSerializer
from sagemaker.deserializers import JSONDeserializer, CSVDeserializer, JSONLinesDeserializer
from sagemaker.explainer import (
    ExplainerConfig, 
    ClarifyExplainerConfig, 
    ClarifyShapConfig, 
    ClarifyInferenceConfig, 
    ClarifyShapBaselineConfig, 
    ClarifyTextConfig
)

jsonlines_serializer = JSONLinesSerializer()
jsonlones_deserializer = JSONLinesDeserializer()

json_serializer = JSONSerializer()
json_deserializer = JSONDeserializer()
csv_serializer = CSVSerializer()
csv_deserializer = CSVDeserializer()
shap_baseline = csv_serializer.serialize([["<UNK>"]])

# {"features": ["hp desk"]}

explainer_config_dict = {
    'ClarifyExplainerConfig': {
        'InferenceConfig': {
            'FeaturesAttribute': 'features',
            'ContentTemplate': '{"features":$features}',
            'LabelAttribute': '[predictions]',
            'ProbabilityAttribute': '[probabilities]',
            'FeatureTypes': ['text'],
            'FeatureHeaders': ['ItemDescription'],
        },
        'ShapConfig': {
            'ShapBaselineConfig': {
                'MimeType': 'text/csv',
                'ShapBaseline': '<UNK>'
            },
            'NumberOfSamples': 5,
            'TextConfig': {
                'Language': 'en', 
                'Granularity': 'sentence'
            }
        }
    }
}
explainer_config_dict

{'ClarifyExplainerConfig': {'InferenceConfig': {'FeaturesAttribute': 'features',
   'ContentTemplate': '{"features":$features}',
   'LabelAttribute': '[predictions]',
   'ProbabilityAttribute': '[probabilities]',
   'FeatureTypes': ['text'],
   'FeatureHeaders': ['ItemDescription']},
  'ShapConfig': {'ShapBaselineConfig': {'MimeType': 'text/csv',
    'ShapBaseline': '<UNK>'},
   'NumberOfSamples': 5,
   'TextConfig': {'Language': 'en', 'Granularity': 'sentence'}}}}

In [58]:
import boto3
boto3_session = boto3.session.Session()
sagemaker_client = boto3.client("sagemaker")

sagemaker_runtime_client = boto3.client("sagemaker-runtime")
sagemaker_session = sagemaker.Session(
    boto_session=boto3_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=sagemaker_runtime_client,
)

version = "inference-toolkit"
endpoint_name = f'nlp-ole-example-with-clarify-{version}'
# sagemaker_client.delete_model(ModelName=endpoint_name)    
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)



In [54]:
from datetime import datetime
import time

import sagemaker
from sagemaker.model import Model
from sagemaker import get_execution_role


print("Current Date and Time:", datetime.now())

role = get_execution_role()
image_uri = f'678264136642.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-studio-d-ebafjgrwlbqa:{version}'

start = time.time()

response = sagemaker_client.create_model(
    ModelName = endpoint_name,
    ExecutionRoleArn = role,
    Containers = [{
        "Image": image_uri,
        "Mode": "SingleModel",
    }]
)

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_name,
    ProductionVariants=[
        {
            "VariantName": "TestVariantName",
            "ModelName": endpoint_name,
            "InitialInstanceCount": 1,
            "InstanceType": 'ml.m4.xlarge',
        }
    ],
    ExplainerConfig=explainer_config_dict,
)

response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_name,
)

sagemaker_session.wait_for_endpoint(endpoint_name)

print("Deployed after mins:", int(time.time() - start) / 60.0)
response

Current Date and Time: 2024-01-24 15:21:49.673393
----!Deployed after mins: 2.533333333333333


{'EndpointArn': 'arn:aws:sagemaker:eu-west-1:123456789012:endpoint/nlp-ole-example-with-clarify-inference-toolkit',
 'ResponseMetadata': {'RequestId': 'c69137b2-b6b7-4244-a2e4-3b8139d542a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c69137b2-b6b7-4244-a2e4-3b8139d542a4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '114',
   'date': 'Wed, 24 Jan 2024 15:21:51 GMT'},
  'RetryAttempts': 0}}

In [55]:
sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

{'EndpointName': 'nlp-ole-example-with-clarify-inference-toolkit',
 'EndpointArn': 'arn:aws:sagemaker:eu-west-1:678264136642:endpoint/nlp-ole-example-with-clarify-inference-toolkit',
 'EndpointConfigName': 'nlp-ole-example-with-clarify-inference-toolkit',
 'ProductionVariants': [{'VariantName': 'TestVariantName',
   'DeployedImages': [{'SpecifiedImage': '678264136642.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-studio-d-ebafjgrwlbqa:inference-toolkit',
     'ResolvedImage': '678264136642.dkr.ecr.eu-west-1.amazonaws.com/sagemaker-studio-d-ebafjgrwlbqa@sha256:c72270e2336706290998fdc6c41410a841d95fc0770ad8c5dd4b3d707dd8600d',
     'ResolutionTime': datetime.datetime(2024, 1, 24, 15, 21, 51, 720000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2024, 1, 24, 15, 21, 51, 74000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 1,

In [57]:
import json
import boto3
import pprint
boto3_session = boto3.session.Session()
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

data = jsonlines_serializer.serialize([{"features": ["hello"]}]).encode('utf-8')

print(f"Calling: {endpoint_name}")
print(f"Payload: {data}")

response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    Body=bytes('{"features": ["hello"]}', 'utf-8')
)

result = json_deserializer.deserialize(response["Body"], content_type=response["ContentType"])
pprint.pprint(result)

Calling: nlp-ole-example-with-clarify-inference-toolkit
Payload: b'{"features": ["hello"]}'
{'explanations': {'kernel_shap': [[{'attributions': [{'attribution': [0.9126373191169496],
                                                      'description': {'partial_text': 'hello',
                                                                      'start_idx': 0}}],
                                    'feature_header': 'ItemDescription',
                                    'feature_type': 'text'}]]},
 'label_headers': ['auto-generated-description-for-hello'],
 'predictions': {'content_type': 'application/json',
                 'data': '{\n'
                         '  "predictions": '
                         '"auto-generated-description-for-hello",\n'
                         '  "probabilities": 0.9160547360268266\n'
                         '}'},
 'version': '1.0'}
